# Clean old Data

In [18]:
import sys
sys.path.insert(0, '../../data/lib/')

import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import CSV

In [19]:
types = {
    'name': 'str',
    'location': 'str',
    'country': 'str',
    'address': 'str',
    'plz': 'str',
    'uci': 'str',
    'type': 'str',
    'source': 'str'
}

# Read pdf into DataFrame
df = pd.read_csv("../../data/0. old data/1. transformation/2015_list.csv", dtype = types)

## Format Table

In [20]:
def revert_name_source(dataset, sep, source):
    for index, row in dataset[(dataset.type == 'hcp') & (dataset.source.isin(source))].iterrows():
        if row['type'] == 'hcp':
            splitted = row['name'].strip().split(sep)
            if len(splitted) == 2:
                dataset.loc[index, 'name'] =  (' '.join(splitted[::-1])).strip()
            else:
                if len(splitted) == 3:
                    dataset.loc[index, 'name'] =  splitted[1] + ' ' + splitted[2] + ' ' + splitted[0]
                else:
                    dataset.loc[index, 'name'] =  (' '.join(splitted[::-1])).strip()
                #dataset.loc[index, '_export_information'] = 'Name besteht aus mehr als drei Wörtern. Evt falsch gedrecht. Bitte checken. Original: ' + row['name']
    
    return dataset

In [21]:
df_export = df.copy()

#Remove rows which have no values
df_export = df_export.dropna(subset=['name', 'location'], how='all')

#basic string conversion
df_export = basic_string_conversion(df_export)

#Country to CH
df_export['country'] = 'CH'

#Add PLZ
df_export = add_plz(df_export)

#Remove text in fields
df_export['name'] = df_export['name'].str.replace('  ', ' ')
df_export['uci'] = df_export['uci'].str.replace('.0', '')
df_export['name'] = df_export['name'].str.replace(', No Title', '')
df_export['name'] = df_export['name'].str.replace(', pract.', '')
df_export['name'] = df_export['name'].str.replace(', lic.', '')
df_export['name'] = df_export['name'].str.replace('Egregio Dott.', '')
df_export['name'] = df_export['name'].str.replace('Egr gio Dott.', '')



#Remove Titles in Name
df_export['name'] = df_export.name.str.replace(regex_title, '', regex=True, case=False)
df_export['name'] = df_export.name.str.strip()

#Remove Comma at the end
df_export['name'] = df_export['name'].str.replace(r',$', '', regex=True)

#Remove comma in otsuka
df_export.loc[df_export.source == 'otsuka', 'name'] = df_export['name'].str.replace(', ', ' ')

#Remove "r."
df_export['name'] = df_export['name'].str.replace(r'\br. ', '', regex=True)

#shift lastname - surname
df_export = revert_name_source(df_export, ' ', ['genzyme', 'msd', 'novartis', 'novo', 'orion', 'sanofi', 'takeda', 'ucb'])
df_export = revert_name_source(df_export, ', ', ['mepha', 'pfizer', 'teva'])



df_export[df_export.address == 'Medizinisch-soziale Hilfestelle1'].head()

#df_export.head()

,name,location,country,address,plz,uci,donations_grants,sponsorship,registration_fees,travel_accommodation,fees,related_expenses,total,type,source
5249,Katharina Hensel-Koch,St. Gallen,CH,Medizinisch-soziale Hilfestelle1,NaN,NaN,NaN,NaN,152.0,518.0,NaN,NaN,670.0,hcp,mundipharma


In [17]:
write_to_excel(df_export, 'tmp.xlsx', open=True)

In [ ]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx')